⚡ Avancé | ⏱ 60 min | 🔑 Concepts : pygame.sprite.Sprite, Group, GroupSingle, collisions

# Sprites et Groupes

## Objectifs

- Comprendre l'intérêt des Sprites par rapport aux Surfaces manuelles
- Créer des classes Sprite personnalisées
- Organiser les Sprites avec les Groupes
- Maîtriser les différentes méthodes de détection de collision
- Structurer le code d'un jeu avec des classes Player, Enemy, Bullet

## Prérequis

- Notions de base de Pygame (notebooks 01-04)
- Compréhension des classes Python (héritage)
- Maîtrise de la boucle de jeu et du delta time

## 1. Introduction aux Sprites

### Pourquoi les Sprites ?

Jusqu'ici, nous avons géré nos objets manuellement : position, image, mise à jour, affichage. Cela fonctionne pour des cas simples, mais devient vite ingérable avec beaucoup d'objets.

**Sans Sprites (manuel)** :
```python
# Gérer chaque ennemi individuellement
ennemis = []
for i in range(50):
    ennemis.append({'x': ..., 'y': ..., 'image': ..., 'vitesse': ...})

# Mise à jour manuelle
for ennemi in ennemis:
    ennemi['y'] += ennemi['vitesse'] * dt
    ecran.blit(ennemi['image'], (ennemi['x'], ennemi['y']))

# Collision manuelle
for ennemi in ennemis:
    for projectile in projectiles:
        if collision_rect(ennemi, projectile):  # Fonction à écrire!
            ...
```

**Avec Sprites** :
```python
# Mise à jour automatique
groupe_ennemis.update(dt)

# Affichage automatique
groupe_ennemis.draw(ecran)

# Collisions en une ligne
collisions = pygame.sprite.groupcollide(groupe_projectiles, groupe_ennemis, True, True)
```

### Avantages des Sprites

| Fonctionnalité | Manuel | Sprites |
|---|---|---|
| Affichage | `blit()` pour chaque objet | `group.draw(ecran)` |
| Mise à jour | Boucle manuelle | `group.update()` |
| Collisions | Calcul manuel | `spritecollide()`, `groupcollide()` |
| Suppression | Retirer des listes | `sprite.kill()` |
| Organisation | Dictionnaires/listes | Classes + Groupes |

## 2. Créer une Classe Sprite

### 2.1 Structure de base

Un Sprite Pygame doit :
1. Hériter de `pygame.sprite.Sprite`
2. Appeler `super().__init__()` dans le constructeur
3. Définir `self.image` (Surface à afficher)
4. Définir `self.rect` (position et dimensions)

```python
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()  # OBLIGATOIRE
        
        # Image du sprite (Surface)
        self.image = pygame.Surface((50, 50))
        self.image.fill((255, 0, 0))
        
        # Rectangle de position
        self.rect = self.image.get_rect()
        self.rect.center = (400, 300)
    
    def update(self):
        # Logique de mise à jour
        pass
```

### 2.2 La méthode update()

La méthode `update()` est appelée automatiquement par le groupe. Elle contient la logique de l'objet :

```python
def update(self, dt):
    # Déplacement
    self.rect.x += self.vitesse_x * dt
    self.rect.y += self.vitesse_y * dt
    
    # Suppression si hors écran
    if self.rect.top > HAUTEUR:
        self.kill()  # Retire le sprite de tous ses groupes
```

### 2.3 kill() : Supprimer un Sprite

`self.kill()` retire le sprite de **tous** les groupes auxquels il appartient. C'est la méthode propre pour supprimer un objet du jeu.

### 2.4 Exemple : Sprite avec image procédurale

```python
class Joueur(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        
        # Créer une image procédurale (triangle)
        self.image = pygame.Surface((40, 40))
        self.image.fill((0, 0, 0))
        self.image.set_colorkey((0, 0, 0))  # Transparence
        pygame.draw.polygon(self.image, (0, 255, 0),
                           [(20, 0), (0, 40), (40, 40)])
        
        # Position
        self.rect = self.image.get_rect(center=(x, y))
        self.vitesse = 300
    
    def update(self, dt):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.vitesse * dt
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.vitesse * dt
        
        # Limiter aux bords
        self.rect.clamp_ip(pygame.Rect(0, 0, LARGEUR, HAUTEUR))
```

## 3. Groupes de Sprites

### 3.1 pygame.sprite.Group

Un `Group` est un conteneur pour sprites. Il offre des méthodes pratiques pour gérer plusieurs sprites en même temps.

```python
# Créer un groupe
groupe = pygame.sprite.Group()

# Ajouter des sprites
sprite1 = MonSprite()
groupe.add(sprite1)

# Ou ajouter directement à la création
sprite2 = MonSprite()
groupe.add(sprite2)

# Nombre de sprites dans le groupe
print(len(groupe))
```

### 3.2 Méthodes principales

| Méthode | Description |
|---|---|
| `group.add(sprite)` | Ajouter un sprite au groupe |
| `group.remove(sprite)` | Retirer un sprite du groupe |
| `group.update(*args)` | Appeler `update()` sur tous les sprites |
| `group.draw(surface)` | Dessiner tous les sprites (utilise `image` et `rect`) |
| `group.empty()` | Retirer tous les sprites |
| `group.sprites()` | Retourner la liste des sprites |
| `len(group)` | Nombre de sprites |

### 3.3 GroupSingle

`GroupSingle` ne contient qu'**un seul sprite** à la fois. Utile pour le joueur :

```python
# Ne contient qu'un sprite à la fois
joueur_group = pygame.sprite.GroupSingle()
joueur_group.add(joueur)

# Accéder au sprite unique
sprite = joueur_group.sprite  # Le sprite ou None
```

### 3.4 Un sprite dans plusieurs groupes

Un même sprite peut appartenir à plusieurs groupes :

```python
tous_les_sprites = pygame.sprite.Group()
ennemis = pygame.sprite.Group()

# L'ennemi est dans les deux groupes
ennemi = Ennemi()
tous_les_sprites.add(ennemi)
ennemis.add(ennemi)

# kill() le retire de TOUS les groupes
ennemi.kill()  # Retiré de tous_les_sprites ET ennemis
```

## 4. Collisions

Pygame fournit plusieurs fonctions de collision dans `pygame.sprite`.

### 4.1 spritecollide() : un sprite vs un groupe

```python
# Vérifier les collisions entre un sprite et un groupe
touches = pygame.sprite.spritecollide(joueur, ennemis, dokill)
# joueur : le sprite à tester
# ennemis : le groupe à tester
# dokill : si True, les sprites touchés sont supprimés du groupe
# Retourne : liste des sprites touchés

# Exemple : le joueur touche un ennemi
touches = pygame.sprite.spritecollide(joueur, ennemis, True)
for ennemi in touches:
    score += 10
```

### 4.2 groupcollide() : groupe vs groupe

```python
# Collisions entre deux groupes
collisions = pygame.sprite.groupcollide(projectiles, ennemis, True, True)
# Premier True : supprimer les projectiles qui touchent
# Deuxième True : supprimer les ennemis touchés
# Retourne : dictionnaire {projectile: [ennemis touchés]}

for projectile, liste_ennemis in collisions.items():
    score += len(liste_ennemis) * 10
```

### 4.3 spritecollideany() : collision rapide

```python
# Retourne le premier sprite touché ou None (plus rapide)
ennemi = pygame.sprite.spritecollideany(joueur, ennemis)
if ennemi:
    print("Collision!")
```

### 4.4 Méthodes de collision

Par défaut, les collisions utilisent les rectangles (`collide_rect`). Mais d'autres méthodes existent :

#### collide_rect (par défaut)

Collision basée sur les rectangles (AABB). Rapide mais imprécis pour les formes non rectangulaires.

```python
# Utilisation par défaut
pygame.sprite.spritecollide(joueur, ennemis, False)

# Explicite
pygame.sprite.spritecollide(joueur, ennemis, False, pygame.sprite.collide_rect)
```

#### collide_rect_ratio

Collision rectangulaire avec un facteur de mise à l'échelle. Utile pour réduire ou agrandir la zone de collision.

```python
# Zone de collision réduite à 80%
pygame.sprite.spritecollide(joueur, ennemis, False,
                            pygame.sprite.collide_rect_ratio(0.8))
```

#### collide_circle

Collision circulaire. Calcule la distance entre les centres.

```python
# Collision basée sur des cercles
pygame.sprite.spritecollide(joueur, ennemis, False,
                            pygame.sprite.collide_circle)

# On peut définir le rayon sur le sprite
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.radius = 25  # Rayon personnalisé
```

#### collide_mask

Collision pixel-perfect. La plus précise, mais la plus coûteuse.

```python
# Collision au pixel près
pygame.sprite.spritecollide(joueur, ennemis, False,
                            pygame.sprite.collide_mask)

# Le sprite doit avoir un attribut mask
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("sprite.png").convert_alpha()
        self.rect = self.image.get_rect()
        self.mask = pygame.mask.from_surface(self.image)
```

### Comparaison des méthodes

| Méthode | Précision | Performance | Usage |
|---|---|---|---|
| `collide_rect` | Basse | Très rapide | Formes rectangulaires |
| `collide_circle` | Moyenne | Rapide | Formes rondes |
| `collide_mask` | Pixel-perfect | Lente | Formes complexes |

## 5. Organisation du Code

### Architecture typique d'un jeu avec sprites

```python
# Classes de sprites
class Player(pygame.sprite.Sprite):
    """Joueur contrôlé au clavier"""
    def __init__(self, x, y):
        super().__init__()
        # ... image, rect, attributs
    
    def update(self, dt):
        # Gestion du mouvement
        pass
    
    def tirer(self):
        # Créer un projectile
        return Bullet(self.rect.centerx, self.rect.top)

class Enemy(pygame.sprite.Sprite):
    """Ennemi qui descend automatiquement"""
    def __init__(self, x, y, vitesse):
        super().__init__()
        # ... image, rect, vitesse
    
    def update(self, dt):
        self.rect.y += self.vitesse * dt
        if self.rect.top > HAUTEUR:
            self.kill()

class Bullet(pygame.sprite.Sprite):
    """Projectile qui monte"""
    def __init__(self, x, y):
        super().__init__()
        # ... image, rect, vitesse
    
    def update(self, dt):
        self.rect.y -= self.vitesse * dt
        if self.rect.bottom < 0:
            self.kill()
```

### Groupes typiques

```python
# Groupes
tous_les_sprites = pygame.sprite.Group()  # Pour draw() et update()
ennemis = pygame.sprite.Group()            # Pour les collisions
projectiles = pygame.sprite.Group()        # Pour les collisions
joueur_group = pygame.sprite.GroupSingle()  # Le joueur

# Créer le joueur
joueur = Player(LARGEUR // 2, HAUTEUR - 50)
tous_les_sprites.add(joueur)
joueur_group.add(joueur)

# Dans la boucle de jeu
tous_les_sprites.update(dt)
tous_les_sprites.draw(ecran)

# Collisions
pygame.sprite.groupcollide(projectiles, ennemis, True, True)
if pygame.sprite.spritecollideany(joueur, ennemis):
    game_over()
```

In [ ]:
%%writefile demo_sprites.py
import pygame
import sys
import random

pygame.init()

# ========== CONSTANTES ==========
LARGEUR, HAUTEUR = 800, 600
FPS = 60

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
VERT = (0, 255, 0)
ROUGE = (255, 0, 0)
JAUNE = (255, 255, 0)
BLEU = (0, 100, 255)

# ========== CLASSES SPRITES ==========
class Joueur(pygame.sprite.Sprite):
    """Vaisseau du joueur contrôlé au clavier"""
    def __init__(self):
        super().__init__()
        # Image procédurale : triangle vert
        self.image = pygame.Surface((40, 40), pygame.SRCALPHA)
        pygame.draw.polygon(self.image, VERT,
                           [(20, 0), (0, 40), (40, 40)])
        
        self.rect = self.image.get_rect()
        self.rect.centerx = LARGEUR // 2
        self.rect.bottom = HAUTEUR - 20
        self.vitesse = 350
    
    def update(self, dt):
        """Déplacer le joueur avec les flèches"""
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.vitesse * dt
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.vitesse * dt
        
        # Limiter aux bords
        self.rect.clamp_ip(pygame.Rect(0, 0, LARGEUR, HAUTEUR))
    
    def tirer(self):
        """Créer un projectile"""
        return Projectile(self.rect.centerx, self.rect.top)


class Ennemi(pygame.sprite.Sprite):
    """Ennemi qui descend depuis le haut"""
    def __init__(self):
        super().__init__()
        # Image procédurale : carré rouge avec croix
        self.image = pygame.Surface((30, 30), pygame.SRCALPHA)
        pygame.draw.rect(self.image, ROUGE, (0, 0, 30, 30))
        pygame.draw.line(self.image, NOIR, (5, 5), (25, 25), 2)
        pygame.draw.line(self.image, NOIR, (25, 5), (5, 25), 2)
        
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, LARGEUR - 30)
        self.rect.y = random.randint(-150, -30)
        self.vitesse = random.randint(100, 250)
    
    def update(self, dt):
        """Descendre et disparaître si hors écran"""
        self.rect.y += self.vitesse * dt
        if self.rect.top > HAUTEUR:
            self.kill()


class Projectile(pygame.sprite.Sprite):
    """Projectile tiré par le joueur"""
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((4, 12), pygame.SRCALPHA)
        pygame.draw.rect(self.image, JAUNE, (0, 0, 4, 12))
        
        self.rect = self.image.get_rect(center=(x, y))
        self.vitesse = 500
    
    def update(self, dt):
        """Monter et disparaître si hors écran"""
        self.rect.y -= self.vitesse * dt
        if self.rect.bottom < 0:
            self.kill()


# ========== INITIALISATION ==========
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Space Invaders - Sprites & Groupes")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

# Groupes de sprites
tous_les_sprites = pygame.sprite.Group()
ennemis = pygame.sprite.Group()
projectiles = pygame.sprite.Group()

# Créer le joueur
joueur = Joueur()
tous_les_sprites.add(joueur)

# Variables de jeu
score = 0
timer_ennemi = 0
intervalle_ennemi = 0.8  # Un ennemi toutes les 0.8 secondes

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    
    # ---- Événements ----
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_SPACE:
                # Tirer un projectile
                projectile = joueur.tirer()
                tous_les_sprites.add(projectile)
                projectiles.add(projectile)
    
    # ---- Génération d'ennemis ----
    timer_ennemi += dt
    if timer_ennemi >= intervalle_ennemi:
        timer_ennemi = 0
        ennemi = Ennemi()
        tous_les_sprites.add(ennemi)
        ennemis.add(ennemi)
    
    # ---- Mise à jour ----
    tous_les_sprites.update(dt)
    
    # ---- Collisions ----
    # Projectiles vs ennemis
    collisions = pygame.sprite.groupcollide(projectiles, ennemis, True, True)
    for projectile, liste_ennemis in collisions.items():
        score += len(liste_ennemis) * 10
    
    # Joueur vs ennemis
    if pygame.sprite.spritecollideany(joueur, ennemis):
        score = max(0, score - 50)
        # Supprimer les ennemis qui touchent le joueur
        touches = pygame.sprite.spritecollide(joueur, ennemis, True)
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Dessiner tous les sprites
    tous_les_sprites.draw(ecran)
    
    # Interface
    score_text = font.render(f"Score: {score}", True, BLANC)
    ennemis_text = font.render(f"Ennemis: {len(ennemis)}", True, BLANC)
    fps_text = font.render(f"FPS: {clock.get_fps():.0f}", True, BLANC)
    info_text = font.render("ESPACE: tirer | Flèches: bouger", True, BLANC)
    
    ecran.blit(score_text, (10, 10))
    ecran.blit(ennemis_text, (10, 45))
    ecran.blit(fps_text, (LARGEUR - 120, 10))
    ecran.blit(info_text, (10, HAUTEUR - 35))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

In [ ]:
%%writefile demo_collisions.py
import pygame
import sys
import random
import math

pygame.init()

# ========== CONSTANTES ==========
LARGEUR, HAUTEUR = 900, 600
FPS = 60

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (60, 60, 60)

# ========== CLASSE BALLE ==========
class Balle(pygame.sprite.Sprite):
    """Balle qui rebondit sur les bords et les autres balles"""
    def __init__(self, x, y, rayon, couleur):
        super().__init__()
        self.rayon = rayon
        self.couleur = couleur
        
        # Image : cercle coloré
        taille = rayon * 2
        self.image = pygame.Surface((taille, taille), pygame.SRCALPHA)
        pygame.draw.circle(self.image, couleur, (rayon, rayon), rayon)
        
        self.rect = self.image.get_rect(center=(x, y))
        
        # Position flottante pour plus de précision
        self.pos_x = float(x)
        self.pos_y = float(y)
        
        # Vitesse aléatoire
        angle = random.uniform(0, 2 * math.pi)
        vitesse = random.uniform(100, 300)
        self.vx = math.cos(angle) * vitesse
        self.vy = math.sin(angle) * vitesse
        
        # Rayon pour collision circulaire
        self.radius = rayon
        
        # Clignotement lors d'une collision
        self.timer_flash = 0
    
    def update(self, dt):
        """Déplacer la balle et rebondir sur les bords"""
        # Déplacer
        self.pos_x += self.vx * dt
        self.pos_y += self.vy * dt
        
        # Rebondir sur les bords horizontaux
        if self.pos_x - self.rayon < 0:
            self.pos_x = self.rayon
            self.vx = abs(self.vx)
        elif self.pos_x + self.rayon > LARGEUR:
            self.pos_x = LARGEUR - self.rayon
            self.vx = -abs(self.vx)
        
        # Rebondir sur les bords verticaux
        if self.pos_y - self.rayon < 0:
            self.pos_y = self.rayon
            self.vy = abs(self.vy)
        elif self.pos_y + self.rayon > HAUTEUR:
            self.pos_y = HAUTEUR - self.rayon
            self.vy = -abs(self.vy)
        
        # Mettre à jour le rect
        self.rect.center = (int(self.pos_x), int(self.pos_y))
        
        # Gérer le flash
        if self.timer_flash > 0:
            self.timer_flash -= dt
            if self.timer_flash <= 0:
                # Restaurer l'image normale
                self.image.fill((0, 0, 0, 0))
                pygame.draw.circle(self.image, self.couleur,
                                  (self.rayon, self.rayon), self.rayon)
    
    def flash(self):
        """Effet visuel lors d'une collision"""
        self.timer_flash = 0.15
        self.image.fill((0, 0, 0, 0))
        pygame.draw.circle(self.image, BLANC,
                          (self.rayon, self.rayon), self.rayon)


def resoudre_collision(balle1, balle2):
    """Résoudre la collision entre deux balles (rebond élastique)"""
    dx = balle2.pos_x - balle1.pos_x
    dy = balle2.pos_y - balle1.pos_y
    distance = math.sqrt(dx * dx + dy * dy)
    
    if distance == 0:
        return
    
    # Vecteur normal
    nx = dx / distance
    ny = dy / distance
    
    # Vitesse relative le long du normal
    dvx = balle1.vx - balle2.vx
    dvy = balle1.vy - balle2.vy
    dvn = dvx * nx + dvy * ny
    
    # Ne pas résoudre si les balles s'éloignent
    if dvn < 0:
        return
    
    # Échanger les composantes normales des vitesses
    balle1.vx -= dvn * nx
    balle1.vy -= dvn * ny
    balle2.vx += dvn * nx
    balle2.vy += dvn * ny
    
    # Séparer les balles pour éviter le chevauchement
    chevauchement = (balle1.rayon + balle2.rayon) - distance
    if chevauchement > 0:
        balle1.pos_x -= chevauchement / 2 * nx
        balle1.pos_y -= chevauchement / 2 * ny
        balle2.pos_x += chevauchement / 2 * nx
        balle2.pos_y += chevauchement / 2 * ny
    
    # Effet visuel
    balle1.flash()
    balle2.flash()


# ========== INITIALISATION ==========
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Collisions circulaires entre balles")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 30)

# Groupe de balles
balles = pygame.sprite.Group()

# Créer des balles initiales
couleurs = [
    (255, 80, 80), (80, 255, 80), (80, 80, 255),
    (255, 255, 80), (255, 80, 255), (80, 255, 255),
    (255, 165, 0), (180, 80, 255), (80, 200, 150)
]

for i in range(12):
    rayon = random.randint(15, 35)
    x = random.randint(rayon + 10, LARGEUR - rayon - 10)
    y = random.randint(rayon + 10, HAUTEUR - rayon - 10)
    couleur = random.choice(couleurs)
    balle = Balle(x, y, rayon, couleur)
    balles.add(balle)

nb_collisions = 0
mode_collision = "circle"  # "rect" ou "circle"

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            # Ajouter une balle avec clic ou espace
            if event.key == pygame.K_SPACE:
                mx, my = pygame.mouse.get_pos()
                rayon = random.randint(15, 35)
                couleur = random.choice(couleurs)
                balle = Balle(mx, my, rayon, couleur)
                balles.add(balle)
            # Changer le mode de collision
            if event.key == pygame.K_TAB:
                mode_collision = "rect" if mode_collision == "circle" else "circle"
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                rayon = random.randint(15, 35)
                couleur = random.choice(couleurs)
                balle = Balle(event.pos[0], event.pos[1], rayon, couleur)
                balles.add(balle)
    
    # Mise à jour
    balles.update(dt)
    
    # Détecter et résoudre les collisions entre balles
    liste_balles = balles.sprites()
    for i in range(len(liste_balles)):
        for j in range(i + 1, len(liste_balles)):
            b1 = liste_balles[i]
            b2 = liste_balles[j]
            
            if mode_collision == "circle":
                # Collision circulaire
                dx = b2.pos_x - b1.pos_x
                dy = b2.pos_y - b1.pos_y
                dist = math.sqrt(dx * dx + dy * dy)
                if dist < b1.rayon + b2.rayon:
                    resoudre_collision(b1, b2)
                    nb_collisions += 1
            else:
                # Collision rectangulaire
                if b1.rect.colliderect(b2.rect):
                    resoudre_collision(b1, b2)
                    nb_collisions += 1
    
    # Affichage
    ecran.fill(NOIR)
    
    # Dessiner les balles
    balles.draw(ecran)
    
    # Dessiner les zones de collision (debug)
    if mode_collision == "circle":
        for balle in balles:
            pygame.draw.circle(ecran, GRIS,
                             (int(balle.pos_x), int(balle.pos_y)),
                             balle.rayon, 1)
    else:
        for balle in balles:
            pygame.draw.rect(ecran, GRIS, balle.rect, 1)
    
    # Interface
    info1 = font.render(f"Balles: {len(balles)} | Collisions: {nb_collisions}", True, BLANC)
    info2 = font.render(f"Mode: {mode_collision} (TAB pour changer)", True, BLANC)
    info3 = font.render("Clic/ESPACE: ajouter une balle", True, BLANC)
    ecran.blit(info1, (10, 10))
    ecran.blit(info2, (10, 40))
    ecran.blit(info3, (10, HAUTEUR - 35))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

## Pièges Courants

### 1. Oublier super().__init__()

```python
# ❌ ERREUR : crash lors de l'ajout au groupe
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        self.image = pygame.Surface((50, 50))
        self.rect = self.image.get_rect()

# ✅ CORRECT
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()  # OBLIGATOIRE!
        self.image = pygame.Surface((50, 50))
        self.rect = self.image.get_rect()
```

### 2. Oublier image ou rect

```python
# ❌ ERREUR : group.draw() plante sans image ou rect
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        # Pas de self.image ni self.rect!

# ✅ CORRECT : les deux sont obligatoires pour draw()
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((50, 50))
        self.image.fill((255, 0, 0))
        self.rect = self.image.get_rect()
```

### 3. Modifier la liste pendant l'itération

```python
# ❌ ERREUR : modifier un groupe pendant une itération
for sprite in groupe:
    if sprite.rect.y > 600:
        groupe.remove(sprite)  # Problème!

# ✅ CORRECT : utiliser kill()
for sprite in groupe:
    if sprite.rect.y > 600:
        sprite.kill()  # Sûr! kill() est différé
```

### 4. Collisions avec dokill mal compris

```python
# ❌ ERREUR : les ennemis sont supprimés sans le vouloir
touches = pygame.sprite.spritecollide(joueur, ennemis, True)
# True supprime les ennemis touchés!

# ✅ CORRECT : False pour ne pas supprimer
touches = pygame.sprite.spritecollide(joueur, ennemis, False)
# Les ennemis restent dans le groupe
```

### 5. collide_mask sans mask

```python
# ❌ ERREUR : pas de mask défini
pygame.sprite.spritecollide(a, b, False, pygame.sprite.collide_mask)
# AttributeError: 'MonSprite' object has no attribute 'mask'

# ✅ CORRECT : définir le mask
class MonSprite(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((50, 50), pygame.SRCALPHA)
        # ... dessiner ...
        self.rect = self.image.get_rect()
        self.mask = pygame.mask.from_surface(self.image)
```

## Mini-Exercices

### Exercice 1 : Étoiles qui tombent

Créez un programme avec :
- Une classe `Etoile(pygame.sprite.Sprite)` qui descend du haut de l'écran
- Les étoiles apparaissent à des positions x aléatoires
- Les étoiles disparaissent quand elles sortent de l'écran (`kill()`)
- De nouvelles étoiles sont créées régulièrement
- Afficher le nombre d'étoiles actives

In [ ]:
# Exercice 1 : À vous de coder!


### Exercice 2 : Ramasser des objets

Créez un jeu où :
- Un joueur (carré) se déplace avec les flèches
- Des pièces (cercles jaunes) apparaissent aléatoirement
- Utiliser `spritecollide()` pour détecter la collecte
- Score +1 par pièce ramassée
- Maximum 10 pièces à l'écran en même temps

In [ ]:
# Exercice 2 : À vous de coder!


### Exercice 3 : Tir et ennemis avec collisions circulaires

Créez un jeu où :
- Le joueur tire des projectiles avec ESPACE
- Des ennemis circulaires descendent
- Utiliser `collide_circle` pour les collisions projectile/ennemi
- Les ennemis ont des tailles différentes (rayon variable)
- Score selon la taille : petit = 30 pts, moyen = 20 pts, grand = 10 pts

In [ ]:
# Exercice 3 : À vous de coder!


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile solution_ex1_etoiles.py
import pygame
import sys
import random

pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 1 : Étoiles qui tombent")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
JAUNE = (255, 255, 0)


class Etoile(pygame.sprite.Sprite):
    """Étoile qui descend du haut de l'écran"""
    def __init__(self):
        super().__init__()
        taille = random.randint(3, 8)
        self.image = pygame.Surface((taille * 2, taille * 2), pygame.SRCALPHA)
        pygame.draw.circle(self.image, JAUNE, (taille, taille), taille)
        
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, LARGEUR)
        self.rect.y = random.randint(-50, -5)
        self.vitesse = random.randint(50, 200)
    
    def update(self, dt):
        self.rect.y += self.vitesse * dt
        # Disparaître si hors écran
        if self.rect.top > HAUTEUR:
            self.kill()


# Groupe
etoiles = pygame.sprite.Group()
timer = 0
intervalle = 0.1  # Nouvelle étoile toutes les 0.1s
total_crees = 0

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Créer des étoiles
    timer += dt
    if timer >= intervalle:
        timer = 0
        etoiles.add(Etoile())
        total_crees += 1
    
    # Mettre à jour
    etoiles.update(dt)
    
    # Dessiner
    ecran.fill(NOIR)
    etoiles.draw(ecran)
    
    info = font.render(f"Étoiles actives: {len(etoiles)} | Total créées: {total_crees}", True, BLANC)
    ecran.blit(info, (10, 10))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 2

In [ ]:
%%writefile solution_ex2_ramasser.py
import pygame
import sys
import random

pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 2 : Ramasser des pièces")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
BLEU = (0, 100, 255)
JAUNE = (255, 215, 0)


class Joueur(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((40, 40))
        self.image.fill(BLEU)
        self.rect = self.image.get_rect(center=(LARGEUR // 2, HAUTEUR // 2))
        self.vitesse = 300
    
    def update(self, dt):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.vitesse * dt
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.vitesse * dt
        if keys[pygame.K_UP]:
            self.rect.y -= self.vitesse * dt
        if keys[pygame.K_DOWN]:
            self.rect.y += self.vitesse * dt
        self.rect.clamp_ip(pygame.Rect(0, 0, LARGEUR, HAUTEUR))


class Piece(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((20, 20), pygame.SRCALPHA)
        pygame.draw.circle(self.image, JAUNE, (10, 10), 10)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(20, LARGEUR - 20)
        self.rect.y = random.randint(20, HAUTEUR - 20)
    
    def update(self, dt):
        pass


# Groupes
tous = pygame.sprite.Group()
pieces = pygame.sprite.Group()

joueur = Joueur()
tous.add(joueur)

# Créer des pièces initiales
for _ in range(10):
    p = Piece()
    tous.add(p)
    pieces.add(p)

score = 0

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    tous.update(dt)
    
    # Collisions joueur/pièces
    ramassees = pygame.sprite.spritecollide(joueur, pieces, True)
    score += len(ramassees)
    
    # Régénérer les pièces
    while len(pieces) < 10:
        p = Piece()
        tous.add(p)
        pieces.add(p)
    
    # Dessiner
    ecran.fill(NOIR)
    tous.draw(ecran)
    
    info = font.render(f"Score: {score} | Pièces: {len(pieces)}", True, BLANC)
    ecran.blit(info, (10, 10))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 3

In [ ]:
%%writefile solution_ex3_tir_collision.py
import pygame
import sys
import random

pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 3 : Tir avec collisions circulaires")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
VERT = (0, 255, 0)
JAUNE = (255, 255, 0)
ROUGE = (255, 60, 60)


class Joueur(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((40, 40), pygame.SRCALPHA)
        pygame.draw.polygon(self.image, VERT, [(20, 0), (0, 40), (40, 40)])
        self.rect = self.image.get_rect(center=(LARGEUR // 2, HAUTEUR - 50))
        self.vitesse = 350
    
    def update(self, dt):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.vitesse * dt
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.vitesse * dt
        self.rect.clamp_ip(pygame.Rect(0, 0, LARGEUR, HAUTEUR))


class Projectile(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((6, 14), pygame.SRCALPHA)
        pygame.draw.rect(self.image, JAUNE, (0, 0, 6, 14))
        self.rect = self.image.get_rect(center=(x, y))
        self.radius = 3  # Pour collide_circle
        self.vitesse = 500
    
    def update(self, dt):
        self.rect.y -= self.vitesse * dt
        if self.rect.bottom < 0:
            self.kill()


class EnnemiCirculaire(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        # Taille aléatoire : petit, moyen, grand
        categorie = random.choice(["petit", "moyen", "grand"])
        if categorie == "petit":
            self.rayon = 12
            self.points = 30
            couleur = (255, 100, 100)
        elif categorie == "moyen":
            self.rayon = 22
            self.points = 20
            couleur = (255, 150, 50)
        else:
            self.rayon = 35
            self.points = 10
            couleur = (200, 50, 50)
        
        taille = self.rayon * 2
        self.image = pygame.Surface((taille, taille), pygame.SRCALPHA)
        pygame.draw.circle(self.image, couleur, (self.rayon, self.rayon), self.rayon)
        
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(self.rayon, LARGEUR - self.rayon)
        self.rect.y = random.randint(-100, -self.rayon * 2)
        self.radius = self.rayon  # Pour collide_circle
        self.vitesse = random.randint(80, 200)
    
    def update(self, dt):
        self.rect.y += self.vitesse * dt
        if self.rect.top > HAUTEUR:
            self.kill()


# Groupes
tous = pygame.sprite.Group()
ennemis = pygame.sprite.Group()
projectiles = pygame.sprite.Group()

joueur = Joueur()
tous.add(joueur)

score = 0
timer_ennemi = 0

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                p = Projectile(joueur.rect.centerx, joueur.rect.top)
                tous.add(p)
                projectiles.add(p)
    
    # Générer des ennemis
    timer_ennemi += dt
    if timer_ennemi >= 0.7:
        timer_ennemi = 0
        e = EnnemiCirculaire()
        tous.add(e)
        ennemis.add(e)
    
    tous.update(dt)
    
    # Collisions circulaires : projectiles vs ennemis
    collisions = pygame.sprite.groupcollide(
        projectiles, ennemis, True, True,
        pygame.sprite.collide_circle
    )
    for proj, liste_e in collisions.items():
        for e in liste_e:
            score += e.points
    
    # Dessiner
    ecran.fill(NOIR)
    tous.draw(ecran)
    
    info = font.render(f"Score: {score} | Ennemis: {len(ennemis)}", True, BLANC)
    ctrl = font.render("ESPACE: tirer | Flèches: bouger", True, BLANC)
    ecran.blit(info, (10, 10))
    ecran.blit(ctrl, (10, HAUTEUR - 35))
    
    pygame.display.flip()

pygame.quit()
sys.exit()